#### PIPE - 8:    
This pipe covers the following : 
1. All features of pipe 7
3. Integration of processing blur.
4. Improved tracker shape and efficiency
5. Integration of Phase 3 for ANRP (Final Phase)
6. Integration of JSON for tracking all the violations

NOTE : We will be also defining the final strucut of json need for capturing violation in the following file

Order of entried in json

violation_type_id - ADDED    
vehicle_type_id - ADDED    
vehicle_speed - ADDED    
travel_direction - ADDED    
location_id - ADDED    
camera_id - ADDED    
evidence_image_url - ADDED    
numbe_plate_ocr_image_url  - ADDED     
evidence_video_url   - ADDED     
numbe_plate_image_url - ADDED    
scanned_number_plate_number - ADDED         
violation_date   - ADDED    
violation_time   - ADDED    

##### Strucutre of JSON
main_json={
"tracking_id":{ and the rest entries}
}

In [1]:
import cv2
import numpy as np
import pandas as pd
import torch
import time
import threading 
from ultralytics import YOLO
from pathlib import Path
from boxmot import StrongSORT
from utils.utils import *


ssd=[
     {'id': 3, 'name': 'car', 'vehicle_class': 'lmv'},
     {'id': 4, 'name': 'motorcycle', 'vehicle_class': 'mcwg/mcg'},
     {'id': 5, 'name': 'truck', 'vehicle_class': 'hgmv'},
     {'id': 6, 'name': 'bus', 'vehicle_class': 'hpmv'},
     {'id': 7, 'name': 'rikshaw', 'vehicle_class': 'l5m'}
    ]
voi=[
   {'id': 2, 'name': 'over_speed'}, 
   {'id': 3, 'name': 'no_helmet'}
]
vehicle_class_json=violation_based_json_creator(ssd)
violation_type_json=violation_based_json_creator(voi)
down = {}
up = {}
text_color = (0, 0, 0)  # Black color for text
yellow_color = (0, 255, 255)  # Yellow color for background
red_color = (0, 0, 255)  # Red color for lines
blue_color = (255, 0, 0)  # Blue color for lines


counter_down = []
counter_up = []

first_boundry_y=170

red_line_y = 198

blue_line_y = 280

offset = 6
# Specify the start and end points of the line
start_point = (300, 198)
end_point = (300, 280)

all_id_tracker=[]
voilation_id_tracker=[]
violation_frame_tracker={}

main_violation_tracker_json={}
x_start=300
x_end=800
y_start=198
y_end=280
#This inputs will be directly taken from front_end
location_id=-1
camera_id=-1
frame_count=0
video_path='/media/hlink/hd/test_videos/testx_vid_1.mp4'

In [2]:
start_time=time.time()
tracker=tracker_init(cuda_device=torch.cuda.is_available(),cuda_device_number=1)
model=YOLO('models/yolov8l.pt')
yolo_model_classes=model.names

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_main = cv2.VideoWriter('output_strongs.avi', fourcc, 20.0, (1020, 500))
all_frames_record_path=create_directory('all_frames_record')
violation_frames_record_path=create_directory('all_violation_record')
current_date=get_current_date()

while True:    
    ret, frame = cap.read()
    
    if not ret:
        break
    
    resized_image=cv2.resize(frame,(1020, 500))
    blurred_image=blur_except_rectangle(resized_image,x_start,y_start,x_end,y_end,offset_size=[30,80],blur_kernel_size=(15,15))
    
    results=model.predict(blurred_image,conf=0.4,verbose=False,device=[0],classes=[2,3])
    # print(f"\n----------------------------------------------------------------------------------\nResult without detection \n{results[0].boxes.data}\n----------------------------------------------------------------------------------\n")
    blurred_image=line_plotter(frame=blurred_image,x_start=300,x_end=774,y=198,line_color=red_color,line_name='boundry 1',line_thickness=2,text_color=text_color)
    blurred_image=line_plotter(frame=blurred_image,x_start=300,x_end=800,y=280,line_color=blue_color,line_name='boundry 2',line_thickness=2,text_color=text_color)
    
    # Specify the color of the line in BGR format (here, it's white)
    color = (255, 255, 255)
    
    # Draw the vertical line on the image
    thickness = 2  # You can adjust the thickness as needed
    cv2.line(blurred_image, start_point, end_point, color, thickness)
    
    # im=cv2.rectangle(blurred_image, (300, 198), (800, 280), (255, 255, 255), 2)
    
    px,conf=prediction_coordinated_hadler(results)
    dets = []
    # Experimenting
    dets,object_class_id=tracker_element_handler(dets,px,conf)        
    
    # print('out_from')
    
    dets = np.array(dets)
    # print(dets)
    if len(dets) > 0:
        tracks = tracker.update(dets, blurred_image) # --> M X (x, y, x, y, id, conf, cls, ind)
        # print('tracks',tracks)    
        for track in tracks:
                travel_direction=-1
                
                blurred_image,object_id,cx,cy,bbox_list,conf=plot_tracks(track,blurred_image)
                if f'{object_id}' in main_violation_tracker_json:
                    pass
                else:
                    main_violation_tracker_json[f'{object_id}']={}
                    # print("adding Id to tracker")
                all_id_tracker.append(object_id)
                
                # Adding file path for csv
                csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'all_frame_detection_detail.csv')
            
                if red_line_y<(cy+offset) and red_line_y > (cy-offset):
                    # print("entered if 1")
                    down[object_id]=time.time()
            
                if object_id in down:
                    # print("entered if 2")
               
                    if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
                         # print("entered if 3")
                         travel_direction=1
                         elapsed_time=time.time() - down[object_id] 
                         # print("entered if 4")
                         if counter_down.count(object_id)==0:
                            counter_down.append(object_id) 
                            distance = 9
                            est_speed=speed_calculator_kmph(distance,elapsed_time)
                            if est_speed>20:
                                
                                # Fucntion for timer tracking
                                
                                time_dict=video_time_checker(count=frame_count,cap=cap)
                                
                                # Getting the name for the detected object
                                object_name=object_class_name_normalizer(object_class_id,yolo_model_classes)
                                
                                                              
                                main_violation_tracker_json[f'{object_id}']['violation_type_id']=violation_type_json['over_speed']
                                main_violation_tracker_json[f'{object_id}']['vehicle_type_id']=vehicle_class_json[f'{object_name}']
                                
                                main_violation_tracker_json[f'{object_id}']['vehicle_speed']=est_speed
                                main_violation_tracker_json[f'{object_id}']['travel_direction']=travel_direction

                                main_violation_tracker_json[f'{object_id}']['location_id']=location_id
                                main_violation_tracker_json[f'{object_id}']['camera_id']=camera_id

                                main_violation_tracker_json[f'{object_id}']['violation_date']=current_date
                                main_violation_tracker_json[f'{object_id}']['violation_time']=time_dict['time']
                                
                                voilation_id_tracker.append(object_id)
                                violation_frame_tracker[object_id]=frame_count
                                # violation_frame_writer_op=frame_writer(frame_count,im,violation_frames_record_path)
                                # print("bbox_list-",bbox_list)
                                object_class_id
                                violation_frame_writer_original=frame_writer(frame_count,resized_image,violation_frames_record_path)
                                violation_frame_writer_blurred=frame_writer(frame_count,resized_image,violation_frames_record_path,coordiante_blur=True,bbox_list=bbox_list)
                                
                                main_violation_tracker_json[f'{object_id}']['evidence_focused_image_url']=violation_frame_writer_blurred
                                main_violation_tracker_json[f'{object_id}']['evidence_image_url']=violation_frame_writer_original
                                            
                                violation_csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'violation_frame_detection_detail.csv')
                                
                            # print(f"for {object_id} speed is -> {est_speed}")
    
    all_frame_writer_op=frame_writer(frame_count,resized_image,all_frames_record_path)    
    frame_count+=1
    out_main.write(resized_image)
    
    # # break on pressing q or spaceq
    cv2.imshow('Strong Sort Detection detection', blurred_image)     
    # key = cv2.waitKey(25) & 0xFF
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
remove_model_from_gpu(model)
end_time=time.time()
print(f'total_time {end_time-start_time}')

2024-05-27 13:33:25.956 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "models/osnet_x0_25_msmt17.pt"


adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
for 3 speed is -> 38
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
for 4 speed is -> 44
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
for 38 speed is -> 34
for 35 speed is -> 26
adding Id to tracker
adding Id to tracker
for 48 speed is -> 31
adding Id to tracker
adding Id to tracker
adding Id to tracker
adding Id to tracker
for 57 speed is -> 33
adding Id to tracker
adding Id to tracker
adding Id to tracker
for 60 speed is -> 25
adding I

In [3]:
%%time
main_violation_json=voilation_capture_json_creator(voilation_id_tracker,csv_file_path)
maintained=json_frame_order_checker(main_violation_json)
if maintained:
    # Function for creating dir paths only
    evidance_img_dir_paths,evidance_clip_dir_paths=evidance_directories_creator(main_violation_json)
    # Function for taking separating image
    evidance_img_dir_paths=evidance_img_separator(evidance_img_dir_paths,all_frames_record_path,main_violation_json)
    
    video_clip_creator_mt(voilation_id_tracker,violation_frame_tracker,evidance_clip_dir_paths,all_frames_record_path,main_violation_tracker_json)
    cropped_images_path_list=evidance_cropper_mt(evidance_img_dir_paths,main_violation_json)
    deblurred_image_paths=deblur_images(cropped_images_path_list,main_violation_json)
    

order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer - True
order_maintainer

In [4]:
%%time
# Using default 8 threads to achive max effiencicy
enhance_image_path=image_enhancement_using_limit_mpx(deblurred_image_paths,num_threads=8)
cropped_lp_paths=preprocssing_part_one(enhance_image_path,'models/LPD.pt',2)

starting thread with id : 0
starting thread with id : 1
CPU times: user 6.04 s, sys: 1.2 s, total: 7.24 s
Wall time: 20min 49s


In [5]:
%%time
enhanced_cropped_images_lp=image_enhancement_using_limit_mpx(cropped_lp_paths,num_threads=8,dir_name='cropped_lp_enhanced')

CPU times: user 517 ms, sys: 1.04 s, total: 1.56 s
Wall time: 24min 27s


In [6]:
enhanced_cropped_images_lp

['/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/enhanced_images_cropped_lp_enhanced/74',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/enhanced_images_cropped_lp_enhanced/57',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/enhanced_images_cropped_lp_enhanced/139',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/enhanced_images_cropped_lp_enhanced/60',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/enhanced_images_cropped_lp_enhanced/92',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/enhanced_images_cropped_lp_enhanced/364',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/enhanced_images_cropped_lp_enhanced/479',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/enhanced_images_cropped_lp_enhanced/483',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/enhanced_images_cropped_lp_enhanced/464',
 '/ho

In [7]:
%%time
thresholded_image=create_threshold_imgs_mp(enhanced_cropped_images_lp)

CPU times: user 6.54 ms, sys: 366 ms, total: 373 ms
Wall time: 768 ms


In [8]:
thresholded_image

['/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/74',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/57',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/139',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/60',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/92',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/364',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/479',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/483',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/464',
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/469',
 '/home/hlink/workspace/learning/boxmot/27_0

In [9]:
from utils.utils import *

In [10]:
def ocr_sub_process_phase_three(folder_paths,csv_file_name,tr_model,tr_processor,tr_device):
        for path in folder_paths:
            path=str(path)
            # print(type(path))
            file_list=sort_files_by_name(path)
            idx=path.split('/')[-1]
            
            for image_path in file_list:
                
                img = cv2.imread(image_path)
                #Currently we write each and every possiblity in the csv file for each Possible ID
                strings=run_ocr(img,tr_model,tr_device,tr_processor)
                final_string=compare_strings(strings)
                
                if valid_license_plate(final_string): 
                    final_csv_string=f'{idx},{final_string},{image_path}'

                else:
                    final_csv_string=f'{idx},{""},{image_path}'

                file_writer(final_csv_string,csv_file_name)

def ocr_processor_mp_phase_three(folder_paths,num_thread=3):
        all_process_list=[]
        chunks=list_divider(folder_paths,number_of_part=num_thread)    
        tr_ocr_models=tr_ocr_model_loader(numthreads=num_thread)
        # main_queue=multiprocessing.Queue()
        current_date=get_current_date()
        csv_file_name=f"{current_date}_ocr_results.csv"
        csv_file_path=os.path.join(os.getcwd(),csv_file_name)
        for chunk,model_json in zip(chunks,tr_ocr_models):
            print(chunk)
            t=threading.Thread(target=ocr_sub_process_phase_three,args=[chunk,csv_file_name,model_json['model'],model_json['processor'],model_json['device']])
            t.start()
            all_process_list.append(t)
        for process in all_process_list:
            process.join()
        return csv_file_path

In [11]:
%%time
lp_csv_path=ocr_processor_mp_phase_three(folder_paths=thresholded_image,num_thread=3)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/74' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/57' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/139'
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/60' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/92' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/364'
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/479' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/483' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/464'
 '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/469' '/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence

/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


<class 'str'>
<class 'str'>


/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


...

<class 'str'>


/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/hlink/workspace/learning/boxmot/bmt/lib/py

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/hlink/workspace/learning/boxmot/bmt/lib/py

CPU times: user 1min 33s, sys: 13 s, total: 1min 46s
Wall time: 1min 45s


In [12]:
# lp_csv_path='/home/hlink/workspace/learning/boxmot/27_05_2024_ocr_results.csv'

In [13]:
# path='/home/hlink/workspace/learning/boxmot/24_05_2024_ocr_results.csv'
ocr_json=ocr_result_filter(main_violation_tracker_json,lp_csv_path)

In [14]:
main_violation_tracker_json=final_json_processor(main_violation_tracker_json,ocr_json)

In [17]:
main_violation_tracker_json=remove_empty_from_dict(main_violation_tracker_json)

In [ ]:
def ocr_result_filter(main_violation_tracker_json,csv_file_path):
    main_json={}
    with open(csv_file_path,'r') as file:
        contents=file.readlines()
        # print(contents)
    splitted_file_contetns=[content.split(',') for content in contents ]
    # print(type(splitted_file_contetns[0][0]))
    for idx in main_violation_tracker_json:
            temp_list=[]
            for count,entry in enumerate(splitted_file_contetns):
                if str(idx)==str(entry[0]):
                    temp_list.append(splitted_file_contetns[count])
            if temp_list:
                main_json[f'{idx}']=temp_list
    return main_json

In [ ]:
def final_json_processor(main_violation_tracker_json,violation_json):
    
    current_dir=os.getcwd()
    fall_back_lp=os.path.join(current_dir,'system_fall_back_assets','fall_back_normal_lp.png')
    fall_back_lp_scanned=os.path.join(current_dir,'system_fall_back_assets','fall_back_scanned_lp.png')
    fall_back_number_plate_number='MH40BP4321'
    str_to_rep='threashold_lp_images'
    scanned_liscense_plate_path=''
    for i in main_violation_tracker_json:
        strings=[]
        if main_violation_tracker_json[i]:    
            main_violation_tracker_json[i]['numbe_plate_ocr_image_url']=fall_back_lp_scanned
            main_violation_tracker_json[i]['numbe_plate_image_url']=fall_back_lp
            main_violation_tracker_json[i]['scanned_number_plate_number']=fall_back_number_plate_number
            if i in violation_json:
                for j in violation_json[i]:
                    if j[1]!='':
                       scanned_liscense_plate_path=j[-1]
                       strings.append(j[1])
                if len(strings)>0:    
                    result=compare_strings(strings)
                    main_violation_tracker_json[i]['numbe_plate_ocr_image_url']=scanned_liscense_plate_path
                    main_violation_tracker_json[i]['numbe_plate_image_url']=scanned_liscense_plate_path.replace(str_to_rep,'enhanced_images_cropped_lp_enhanced').replace('\n','')
                    main_violation_tracker_json[i]['scanned_number_plate_number']=result 
            
    return main_violation_tracker_json

In [ ]:
demo='/home/hlink/workspace/learning/boxmot/27_05_2024_infenrence_output/threashold_lp_images/375/10765.jpg\n'

In [ ]:
str_to_rep='threashold_lp_images'
replaced=demo.replace(str_to_rep,'enhanced_images_cropped_lp_enhanced').replace('\n','')
print(replaced)
print(os.path.exists(replaced))